# Librerías y configuraciones


In [45]:
import pandas as pd

In [46]:
pd.set_option('display.max_colwidth', 100)

---

# Carga de datos

In [47]:
df_genres = pd.read_csv('data/df_genres.csv')
df_item_genre = pd.read_csv('data/df_item_genre.csv')
df_items = pd.read_csv('data/df_items.csv')
df_reviews = pd.read_csv('data/df_reviews.csv')
df_users_items = pd.read_csv('data/df_users_items.csv')
df_users = pd.read_csv('data/df_users.csv')
df_user_id_item_id_price = pd.read_csv("data/df_user_id_item_id_price.csv")
df_developer = pd.read_csv('data/df_developer.csv')

In [48]:
def check(input,df,columna):
    return any(part in input.lower() for part in df[columna])

----

1. def userdata( User_id : str ): 
    Debe devolver 
    * cantidad de dinero gastado por el usuario,  
    * el porcentaje de recomendación en base a reviews.recommend
    * cantidad de items.


In [49]:
def userdata(user_id: str):
    # Si el usuario no existe devuelve: {'dinero_gastado': 0.0, 'porcentaje_de_recomendaciones': nan, 'items_recomendados': 0, 'items_comprados': 0}
    
    user_id = user_id.strip()

    # Dinero gastado
    df_filtrado_dinero = df_user_id_item_id_price[df_user_id_item_id_price.user_id == user_id]
    dinero_gastado = round(df_filtrado_dinero.price.sum(),2)

    # Porcentaje de recomendaciones
    df_filtrado_recomendaciones = df_reviews[df_reviews.user_id == user_id]
    
    if df_filtrado_recomendaciones.shape[0] == 0:
         porcentaje_de_recomendacion = 0
    else:
        porcentaje_de_recomendacion = round(df_filtrado_recomendaciones.recommend.mean(),3)*100
    
    # items comprados
    items_comprados = int(df_filtrado_dinero.shape[0])
    
    # items recomendados
    items_recomendados = int(df_filtrado_recomendaciones.shape[0])

    out = {"dinero_gastado":dinero_gastado, "porcentaje_de_recomendaciones": porcentaje_de_recomendacion, 'items_recomendados': items_recomendados,'items_comprados':items_comprados}

    return out

In [50]:
userdata('0212030')

{'dinero_gastado': 0.0,
 'porcentaje_de_recomendaciones': 0,
 'items_recomendados': 0,
 'items_comprados': 0}

In [51]:
df_user_id_item_id_price[df_user_id_item_id_price.user_id == '0212030']

,user_id,item_id,price


In [52]:
df_reviews[df_reviews.user_id == '0212030']

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review,posted_date


----

2. def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): 
    * Cantidad de usuarios que realizaron reviews entre las fechas dadas y, 
    * el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [53]:
####### --------------->>>  Función 2

def countreviews (fecha_inicio:(str), fecha_fin):

    df_filtrado = df_reviews[(df_reviews['posted_date'] >= fecha_inicio) & (df_reviews['posted_date'] <= fecha_fin)]
    cant_usuarios = df_filtrado.shape[0]
    porcentaje_de_recomendacion = round(df_filtrado.recommend.mean(),3)*100
    return {"cantidad":cant_usuarios, "porcentaje":porcentaje_de_recomendacion}
    # return df_filtrado

In [54]:
# pruebas
countreviews('2011-11-05' , '2011-11-20')

{'cantidad': 11, 'porcentaje': 90.9}

In [55]:
# el df df_reviews tiene valores nulos en la columna posted_date. Revisar como influyen en la funcion, si los toma en cuenta, si los ignora, etc.
#  se podria agregar un verificador de fechas validas.

---

3. def genre( género : str ): 
    * Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [56]:
####### --------------->>>  Función 3

def genre (genero):
    genero_minusculas = genero.lower().strip()
    if genero_minusculas in list(df_genres.name):
        puesto = int(df_genres[df_genres.name == genero_minusculas].reset_index().at[0,'ranking'])
        salida = {"genero": genero_minusculas, "ranking":puesto}
    else:
        puesto = 'No se encuentra. Intente nuevamente. Ej: Action'
        salida = {"genero": genero_minusculas, "ranking":puesto}
    return salida

In [57]:
# Pruebas
print(genre ('strategy'))
print(genre ('action '))
print(genre ('    ACTION'))
print(genre ('strateggggy'))

{'genero': 'strategy', 'ranking': 6}
{'genero': 'action', 'ranking': 1}
{'genero': 'action', 'ranking': 1}
{'genero': 'strateggggy', 'ranking': 'No se encuentra. Intente nuevamente. Ej: Action'}


----


4.
def userforgenre( género : str ): 
        games.genres
        games.id (item_id)
    Top 5 de usuarios
        head(5)
    con más horas de juego en el género dado, 
        item_df.items.playtime_forever
    con su URL (del user) y 
        items_df.user_url
    user_id.
        items_df.user_id

        *tabla genre
        *tabla genre_item
        *tabla items:        
        *tabla user_item:
        *tabla user:

In [58]:
df_item_genre.head(2)

,item_id,genres
0,761140,action
1,761140,casual


In [59]:
df_users_items.head(2)

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,30,7


In [60]:
df_users.head(2)

,Unnamed: 0,user_id,items_count,user_url
0,0,76561197970982479,277,http://steamcommunity.com/profiles/76561197970982479
1,1,js41637,888,http://steamcommunity.com/id/js41637


In [61]:
# -->>>> funcion 4

def userforgenre( genero : str ): 
    genero_min = genero.lower().strip()

    if genero_min in list(df_item_genre.genres):

        lista_de_items_del_genero = list(df_item_genre[df_item_genre.genres == genero_min].item_id)
        df = df_users_items[df_users_items.item_id.isin(lista_de_items_del_genero)]
        top5 = df.groupby('user_id')['playtime_forever'].sum().reset_index()
        top5 = top5.sort_values('playtime_forever',ascending=False).head() # aca ta el user id y las hs de juego
        top5 = top5.merge(df_users[['user_id','user_url']], on='user_id', how='left')
        users_top5 = list(top5.user_id)
        user_url_top5 = list(top5.user_url)

        return {"users_top5":users_top5, "user_url_top5":user_url_top5}

    else:
        mensaje = 'el género ', genero, ' no éxiste. Intente nuevamente. EJ: "Action"'
        return mensaje

In [62]:
userforgenre('action')

{'users_top5': ['Sp3ctre',
  'shinomegami',
  'REBAS_AS_F-T',
  'Terminally-Chill',
  'DownSyndromeKid'],
 'user_url_top5': ['http://steamcommunity.com/id/Sp3ctre',
  'http://steamcommunity.com/id/shinomegami',
  'http://steamcommunity.com/id/REBAS_AS_F-T',
  'http://steamcommunity.com/id/Terminally-Chill',
  'http://steamcommunity.com/id/DownSyndromeKid']}

-----

5.
def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:
            Ejemplo de salida:
            Activision	
            Año	Contenido Free
            2023	27%
            2022	25%
            xxxx	xx%

            *tabla games
            *tabla games_tags
            *tabla tags

In [63]:
df_developer.head(2)

,developer,price,year
0,kotoshiro,4.99,2018
1,secret level srl,0.00,2018


In [74]:
def developer(desarrollador : str ):

    desarrollador_minusculas = desarrollador.lower().strip()

    if check(desarrollador_minusculas,df_developer,'developer'):
        
        df_filtrado = df_developer[df_developer.developer == desarrollador_minusculas]


        df_xanio = df_filtrado.groupby(['year']).size().reset_index(name=('cantidad_total'))
        df_filtrado_free = df_filtrado[df_filtrado.price == 0].groupby(['year']).size().reset_index(name=('cantidad_free'))
        df = df_xanio.merge(df_filtrado_free,how='left')
        cantidad_de_items = df.shape[0]
        df['porcentaje'] = round(df.cantidad_free /df.cantidad_total,3) *100
        df['porcentaje'] = df['porcentaje'].fillna(0)


        anios = df_xanio['year'].tolist()
        porcentaje_free_por_anio = df['porcentaje'].tolist()

        out = {
            "cantidad_de_items": cantidad_de_items,
            "anios": anios,
            "porcentaje_free_por_anio": porcentaje_free_por_anio
            }
        
    else:
         out = (F'El desarrollador {desarrollador}, no éxiste. Intente nuevamente. EJ: "kotoshiro"')

    return out

In [75]:
developer('sacada')

{'cantidad_de_items': 2,
 'anios': [2017, 2018],
 'porcentaje_free_por_anio': [0.0, 0.0]}

In [72]:
df_developer

,developer,price,year
0,kotoshiro,4.99,2018
1,secret level srl,0.00,2018
2,poolians.com,0.00,2017
3,彼岸领域,0.99,2017
4,trickjump games ltd,3.99,2018
...,...,...,...
27588,bidoniera games,1.99,2018
27589,"nikita ""ghost_rus""",1.99,2018
27590,sacada,4.99,2018
27591,laush dmitriy sergeevich,1.99,2018


In [67]:
df_filtrado = df_developer[df_developer.developer == 'sacada']

df_xanio = df_filtrado.groupby(['year']).size().reset_index(name=('cantidad_total'))
df_filtrado_free = df_filtrado[df_filtrado.price == 0].groupby(['year']).size().reset_index(name=('cantidad_free'))
df = df_xanio.merge(df_filtrado_free,how='left')
df['porcentaje'] = round(df.cantidad_free /df.cantidad_total,3) *100
df['porcentaje'] = df['porcentaje'].fillna(0)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            2 non-null      int64  
 1   cantidad_total  2 non-null      int64  
 2   cantidad_free   0 non-null      float64
 3   porcentaje      2 non-null      float64
dtypes: float64(2), int64(2)
memory usage: 80.0 bytes


In [69]:
df_items

,item_id,title,url,release_date,developer,price
0,761140,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/,2018-01-04,Kotoshiro,4.99
1,643980,Ironbound,http://store.steampowered.com/app/643980/Ironbound/,2018-01-04,Secret Level SRL,0.00
2,670290,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_Pool_3D__Poolians/,2017-07-24,Poolians.com,0.00
3,767400,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,彼岸领域,0.99
4,773570,NaN,http://store.steampowered.com/app/773570/Log_Challenge/,NaN,NaN,2.99
...,...,...,...,...,...,...
30749,773640,Colony On Mars,http://store.steampowered.com/app/773640/Colony_On_Mars/,2018-01-04,"Nikita ""Ghost_RUS""",1.99
30750,733530,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGistICAL_South_Africa/,2018-01-04,Sacada,4.99
30751,610660,Russian Roads,http://store.steampowered.com/app/610660/Russian_Roads/,2018-01-04,Laush Dmitriy Sergeevich,1.99
30752,658870,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_2__Directions/,2017-09-02,"xropi,stev3ns",4.99
